Вывод текста везде (кроме tika) закомменчен для удобства скроллинга

В данном ноутбуке рассмотрены несколько библиотек для извлечения текста из pdf, doc, docx и rtf файлов

Рассмотренные библиотеки: Apache Tika, PyPDF2, PyMuPDF (fitz), pdfminer.six, python-docx, docx2txt и способ чтения без дополнительных библиотек; так же рассмотрен способ конвертации файлов docx->pdf с помощью библиотеки docx2pdf

In [2]:
!pip install ipython-autotime
%load_ext autotime
import warnings
warnings.filterwarnings("ignore")

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.63 s (started: 2023-01-30 13:09:43 +00:00)


# Apache Tika
Для работы обязательно нужны установленная Java SE Development Kit 19.0.1 (по требованиям версия 1.6 и выше) и jar-файл (лежит в директории src); ссылка на <a href="https://www.oracle.com/java/technologies/javase/jdk19-archive-downloads.html">java</a>

Перед запуском функции get_document_text необходимо запустить с помощью java jar-файл C:\Users\username\AppData\Local\Temp\tika-server.jar

Команда для запуска в командной строке java -jar C:\Users\username\AppData\Local\Temp\tika-server.jar

После выполнения все должно работать

Принимаемые функцией get_document_text расширения файлов: .doc, .docx, .pdf, .rtf; Но вообще tika способна работать с практически любыми расширениями файлов (медиа, текст и т.д.)

После окончания работы с tika необходимо выключить открытый jar-файл (для этого вроде как достаточно закрыть cmd)

In [3]:
import tika
tika.initVM()

time: 17.2 ms (started: 2023-01-30 13:09:45 +00:00)


In [4]:
from tika import parser

def get_document_text(filepath):
    doc_type = filepath.split('.')[-1]
    if doc_type in ['doc', 'rtf', 'pdf', 'docx']:
        text = parser.from_file(filepath)['content']
    else:
        print(f'File extention must be either pdf, rtf, doc or docx, got {doc_type}')
    return text.strip()

time: 461 ms (started: 2023-01-30 13:09:45 +00:00)


In [6]:
path_doc="../data/parsed_data/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.doc"
print(get_document_text(path_doc)[:1000])

Образец простого договора аренды квартиры, заключаемого между физическим лицами

ДОГОВОР НАЙМА КВАРТИРЫ
	г. _______________
	«____» ______________ 2022 г.


 Гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наймодатель», с одной стороны, и гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наниматель», с другой стороны, именуемые в дальнейшем «Стороны», заключили настоящий договор, в дальнейшем «Договор», о нижеследующем: 
1. Наймодатель предоставляет Нанимателю во временное владение и пользование за плату принадлежащую ему по праву собственности квартиру, кадастровый номер: ________ , находящуюся по адресу: ______________________________________

In [7]:
path_docx="../data/parsed_data/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx"
print(get_document_text(path_docx)[:1000])

Образец простого договора аренды квартиры, заключаемого между физическим лицами

ДОГОВОР НАЙМА КВАРТИРЫ
	г. _______________
	«____» ______________ 2022 г.





 Гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наймодатель», с одной стороны, и гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наниматель», с другой стороны, именуемые в дальнейшем «Стороны», заключили настоящий договор, в дальнейшем «Договор», о нижеследующем: 
1. Наймодатель предоставляет Нанимателю во временное владение и пользование за плату принадлежащую ему по праву собственности квартиру, кадастровый номер: ________ , находящуюся по адресу: ___________________________________

In [12]:
path_pdf="../data/parsed_data/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf"
print(get_document_text(path_pdf)[:1000])

Образец договора аренды автомобильного прицепа, заключаемого между юридическим и физическим лицом


ДОГОВОР АРЕНДЫ&NBSP;ДОГОВОР АРЕНДЫ&NBSP;
автомобильного прицепаавтомобильного прицепа

г. _______________г. _______________ «_____» _______________ 2023 г.«_____» _______________ 2023 г.

________________________________________________ в лице________________________________________________ в лице
________________________________________________, действующего на основании________________________________________________, действующего на основании
________________________________________________, именуемый в дальнейшем «________________________________________________, именуемый в дальнейшем «АрендодательАрендодатель», с», с
одной стороны, и гр. ________________________________________________, паспорт: серия ________, №одной стороны, и гр. ________________________________________________, паспорт: серия ________, №
________, выданный ________________________, проживающий по адресу:_______

## **Библиотека PyPDF2.**
Работает только с PDF-файлами

In [19]:
!pip install pypdf2
from PyPDF2 import PdfReader

time: 1.49 s (started: 2023-01-30 13:13:53 +00:00)


Время чтения и вывода pdf-файла размером 11 страниц - 10.8 с, но при этом шрифт с тенью выводится в единственном экземпляре (+)

In [45]:
pdf_pypdf2 = '../data/parsed_data/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf'
text = []
with open(pdf_pypdf2, "rb") as filehandle:  
    pdf = PdfReader(filehandle)
    pages = len(pdf.pages)
    for i in range(pages):
        page = pdf.pages[i]
        text.append(page.extract_text())
print(text[0][:1000])

ДОГОВОР АРЕНДЫ&NBSP;ДОГОВОР АРЕНДЫ&NBSP;
автомобильного прицепаавтомобильного прицепа
г. _______________г. _______________ «_____» _______________ 2023 г.«_____» _______________ 2023 г.
________________________________________________ в лице________________________________________________ в лице
________________________________________________, действующего на основании________________________________________________, действующего на основании
________________________________________________, именуемый в дальнейшем «________________________________________________, именуемый в дальнейшем « АрендодательАрендодатель », с», с
одной стороны, и гр. ________________________________________________, паспорт: серия ________, №одной стороны, и гр. ________________________________________________, паспорт: серия ________, №
________, выданный ________________________, проживающий по адресу:________, выданный ________________________, проживающий по адресу:
_______________________________________

## **PyMuPDF (он же fitz)**
Работает с форматами **PDF**, XPS, OpenXPS, CBZ, EPUB и FB2
<p>Чувствителен к теням у шрифта

In [46]:
!pip install pymupdf
import fitz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 8.5 MB/s eta 0:00:00:00:0100:01
time: 4.54 s (started: 2023-01-30 13:19:57 +00:00)


Время чтения и вывода pdf-файла размером 11 страниц - 53 мс

In [48]:
pdf_fitz = "../data/parsed_data/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf"
text = ''
with fitz.open(pdf_fitz) as doc:
    for page in doc:
        text += page.get_text()
#Из-за тени в исходном документе происходит дублирование, данная строка выделяет одну из двух
text = '\n'.join(text.split('\n')[::2])
print(text[:1000])

ДОГОВОР АРЕНДЫ&NBSP;
автомобильного прицепа
г. _______________
«_____» _______________ 2023 г.
________________________________________________ в лице
________________________________________________, действующего на основании
________________________________________________, именуемый в дальнейшем «
Арендодатель», с
одной стороны, и гр. ________________________________________________, паспорт: серия ________, №
________, выданный ________________________, проживающий по адресу:
________________________________________________, именуемый в дальнейшем «
Арендатор», с
другой стороны, именуемые в дальнейшем «Стороны», заключили настоящий договор, в
дальнейшем «
Договор», о нижеследующем: 
1. ПРЕДМЕТ ДОГОВОРА
1.1. По настоящему договору Арендодатель предоставляет Арендатору во временное
пользование за плату прицеп для легкового автомобиля модели ________________________
(включая дополнительное оборудование, установленное на прицеп), далее именуемый
«Прицеп».
Государственный регистрационны

В fitz можно раскрашивать конкретные слова

In [49]:
text = ''
highlight_text = ['Договор','Квартир']
doc = fitz.open(pdf_fitz)
for current_page in doc:
    for word in highlight_text:
        text_instances = current_page.search_for(word)
        for inst in text_instances:
            highlight = current_page.add_highlight_annot(inst)
            highlight.set_colors({"stroke":(0, 0.5, 0)})
            highlight.update()
doc.save("../data/parsed_data/Договор_аренды_автомобильного_прицепа_colored.pdf", garbage=4, deflate=True, clean=True)

time: 63.4 ms (started: 2023-01-30 13:20:56 +00:00)


## pdfminer.six (версия pdfminer для python 3.*)
Работа только с pdf-файлами
<p> Чувствителен к теням

In [50]:
!pip install pdfminer.six
from pdfminer.high_level import extract_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 7.3 MB/s eta 0:00:0000:0100:01
time: 3.52 s (started: 2023-01-30 13:21:02 +00:00)


Время чтения и вывода pdf-файла размером 11 страниц - 2.18 с

In [51]:
pdf_pdfminer = '../data/parsed_data/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf'
text_pdfminer = extract_text(pdf_pdfminer)
#Из-за тени в исходном документе происходит дублирование, данная строка выделяет одну из двух
text_pdfminer = '\n'.join(text_pdfminer.split('\n')[::2])
print(text_pdfminer[:1000])

The PDF <_io.BufferedReader name='../data/parsed_data/Договоры аренды/Договор_аренды_автомобильного_прицепа.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


ДОГОВОР АРЕНДЫ&NBSP;

автомобильного прицепа
г. _______________

«_____» _______________ 2023 г.
________________________________________________ в лице
________________________________________________, действующего на основании
Арендодатель», с», с
________________________________________________, именуемый в дальнейшем «
одной стороны, и гр. ________________________________________________, паспорт: серия ________, №
________, выданный ________________________, проживающий по адресу:
________________________________________________, именуемый в дальнейшем «Арендатор
другой стороны, именуемые в дальнейшем «Стороны», заключили настоящий договор, в
дальнейшем «Договор

Договор», о нижеследующем: 
1. ПРЕДМЕТ ДОГОВОРА

1.1. По настоящему договору Арендодатель предоставляет Арендатору во временное
пользование за плату прицеп для легкового автомобиля модели ________________________
(включая дополнительное оборудование, установленное на прицеп), далее именуемый
«Прицеп».
Государственный реги

## **Чтение docx файлов**

## **python-docx (docx)**
Работа только с docx

In [52]:
!pip install python-docx
import docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 6.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 9.1 MB/s eta 0:00:00:00:0100:01
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184490 sha256=c7b41bf4834cedcd16bbdfbaa1a2f05f69cba5535830dfb8a787ba798d5c0f4c
  Stored in directory: /home/jovyan/.cache/pip/wheels/65/e1/9b/0c38fe6cfe02a9fe31cb6b4efd90985f17354d7f77872f2def
Successfully built python-docx
time: 6.8 s (started: 2023-01-30 13:21:21 +00:00)


Не смог разобраться, почему не работает

In [53]:
doc = '../data/parsed_data/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx'
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)
getText(doc)[:1000]

XMLSyntaxError: Start tag expected, '<' not found, line 1, column 1 (<string>, line 1)

time: 22.1 ms (started: 2023-01-30 13:21:30 +00:00)


## **docx2txt**
Работает только с форматом docx простым переводом в txt

In [54]:
!pip install docx2txt 
import docx2txt

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3961 sha256=36f3b0fe5096091740b0525bca5a99b7f4b0b4320f394300dff412ab135cfc3a
  Stored in directory: /home/jovyan/.cache/pip/wheels/3c/78/a3/ef70996aa66e38229494dbd84cd747e2e0668f1940bf4a7de8
Successfully built docx2txt
time: 3.24 s (started: 2023-01-30 13:21:36 +00:00)


Сохраняет форматирование, время чтения и вывода - 18 мс

In [56]:
text = docx2txt.process("../data/parsed_data/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx")
print(text[:1000])

ДОГОВОР НАЙМА КВАРТИРЫ

г. _______________

«____» ______________ 2022 г.







 Гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наймодатель», с одной стороны, и гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наниматель», с другой стороны, именуемые в дальнейшем «Стороны», заключили настоящий договор, в дальнейшем «Договор», о нижеследующем: 

1. Наймодатель предоставляет Нанимателю во временное владение и пользование за плату принадлежащую ему по праву собственности квартиру, кадастровый номер: ________ , находящуюся по адресу: ________________________________________________ , общая площадь ________ кв.м, состоящую из ________ комнат (дал

## **Простой метод, использующий стандартную библиотеку zipfile**
Для работы с docx файлами

In [57]:
try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML
import zipfile

time: 31 ms (started: 2023-01-30 13:21:59 +00:00)


Основан на python-docx, не требует дополнительной установки библиотек, простая функция для чтения, время чтения и вывода - 11.7 мс

In [58]:
WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = WORD_NAMESPACE + 'p'
TEXT = WORD_NAMESPACE + 't'


def get_docx_text(path):
    """
    Take the path of a docx file as argument, return the text in unicode.
    """
    document = zipfile.ZipFile(path)
    xml_content = document.read('word/document.xml')
    document.close()
    tree = XML(xml_content)

    paragraphs = []
    for paragraph in tree.iter(PARA):
        texts = [node.text
                 for node in paragraph.iter(TEXT)
                 if node.text]
        if texts:
            paragraphs.append(''.join(texts))

    return '\n\n'.join(paragraphs)

time: 616 µs (started: 2023-01-30 13:22:01 +00:00)


In [59]:
docx = '../data/parsed_data/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx'
print(get_docx_text(docx)[:1000])

ДОГОВОР НАЙМА КВАРТИРЫ

г. _______________

«____» ______________ 2022 г.

 Гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наймодатель», с одной стороны, и гр. ________________________________________________, паспорт: серия ________, № ________, выданный ________________________, проживающий по адресу: ________________________________________________, именуемый в дальнейшем «Наниматель», с другой стороны, именуемые в дальнейшем «Стороны», заключили настоящий договор, в дальнейшем «Договор», о нижеследующем: 

1. Наймодатель предоставляет Нанимателю во временное владение и пользование за плату принадлежащую ему по праву собственности квартиру, кадастровый номер: ________ , находящуюся по адресу: ________________________________________________ , общая площадь ________ кв.м, состоящую из ________ комнат (далее Ква

## Aspose-words
Платная библиотека (в бесплатной версии остается водяной знак и ограничен функционал)
<p> Работает с doc, doc, odt, pdf, html </p>
<p> Чувствителен к теням

In [64]:
!pip install aspose-words
import aspose.words as aw


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 MB 4.9 MB/s eta 0:00:00:00:0100:01
time: 15.5 s (started: 2023-01-30 13:23:13 +00:00)


In [ ]:
path1="../data/parsed_data/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.doc"

doc = aw.Document(path1)
saveOptions = aw.saving.PdfSaveOptions()
saveOptions.compliance = aw.saving.PdfCompliance.PDF17 
doc.save("../data/parsed_data/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.pdf", saveOptions)
pdf_fitz = "../data/parsed_data/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.pdf"

doc = fitz.open(pdf_fitz)
text = ''
for current_page in range(doc.page_count):
    text += doc.get_page_text(current_page,"text")
#Из-за тени в исходном документе происходит дублирование, данная строка выделяет одну из двух
text = '\n'.join(text.split('\n')[::2])
print(text[:1000])

## Конвертер docx2pdf
Хорошо, но достаточно долго конвертирует docx и doc файлы в pdf

In [61]:
!pip install docx2pdf
from docx2pdf import convert

time: 1.55 s (started: 2023-01-30 13:22:39 +00:00)


In [62]:
convert("../data/parsed_data/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.docx", "../data/parsed_data/Образец_простого_договора_аренды_квартиры-_заключаемого_между_физическим_лицами.pdf")

NotImplementedError: docx2pdf is not implemented for linux as it requires Microsoft Word to be installed

time: 27 ms (started: 2023-01-30 13:22:40 +00:00)
